In [1]:
import pandas as pd
import xlrd
import numpy as np 
import regex as re
import string
import csv
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud
import matplotlib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import MiniBatchKMeans

In [2]:
dfs = pd.read_excel('Company Descriptions.xlsx', sheet_name='Sheet1')

In [3]:
dfs.head()

,company_name,company_short_description,company_description
0,Codementor,Codementor is an online marketplace connecting...,Codementor provides live 1:1 help for software...
1,AgShift,AgShift is designing world's most advanced aut...,AgShift solution blends Deep Learning with Com...
2,Shipsi,Shipsi empowers any retailer with the ability ...,Shipsi empowers any retailer with the ability ...
3,OpenNews,"OpenNews helps a global network of developers,...","We're helping a global network of developers, ..."
4,Biobot Analytics,Biobot Analytics analyzes city sewage to estim...,Biobot Analytics analyzes sewage to estimate o...


In [4]:
print(len(dfs))

19965


In [5]:
comp_disc = dfs.company_description.astype('U')
comp_disc.head()

0    Codementor provides live 1:1 help for software...
1    AgShift solution blends Deep Learning with Com...
2    Shipsi empowers any retailer with the ability ...
3    We're helping a global network of developers, ...
4    Biobot Analytics analyzes sewage to estimate o...
Name: company_description, dtype: object

In [6]:
tfidf = TfidfVectorizer(min_df = 5, max_df = 0.95, max_features = 8000, stop_words = 'english')
tfidf.fit(comp_disc)
text_token = tfidf.transform(comp_disc)

In [7]:
clusters = MiniBatchKMeans(n_clusters=4, init_size=1024, batch_size=2048, random_state=20).fit_predict(text_token)

In [8]:
def top_keywords(data, clusters, labels, n_terms):
    df = pd.DataFrame(data.todense()).groupby(clusters).mean()
    
    for i,r in df.iterrows():
        print('\nCluster {}'.format(i))
        print(','.join([labels[t] for t in np.argsort(r)[-n_terms:]]))
            
top_keywords(text_token, clusters, tfidf.get_feature_names(), 10)


Cluster 0
people,new,world,based,com,online,technology,mobile,company,platform

Cluster 1
based,service,company,business,provides,management,software,services,solutions,data

Cluster 2
esports,espn,especially,escape,errors,error,erp,established,zynga,nan

Cluster 3
diseases,development,therapeutics,care,clinical,health,patients,treatment,medical,company


In [9]:
wb = xlrd.open_workbook("Company Descriptions.xlsx")   
sheet = wb.sheet_by_index(0)
row_list = [["company","cluster"]]
for i in range(1,19966):
    j=clusters[i-1]
  #sheet1.cell_value(j+1, 0)
    row_list.append([sheet.cell_value(i,0),j])
with open('ISB_Task_3_2.csv', 'w', newline='', encoding ="utf8") as file:
    writer = csv.writer(file)
    writer.writerows(row_list)